# Zadanie 3

Zadanie zostało zrobione na podstawie kursu z datacamp : https://campus.datacamp.com/courses/intermediate-deep-learning-with-pytorch/images-convolutional-neural-networks?ex=5 
Oraz artykułu cs231 : https://cs231n.github.io/convolutional-networks/?utm_source=chatgpt.com

Używam Clouds dataset ( 7 klas ) - z kaggla

https://www.kaggle.com/datasets/jockeroika/clouds-photos/data

Libraries

In [187]:
import kagglehub
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from torchmetrics import Precision, Recall
import torch
import random
import numpy as np

Downloading data

In [188]:

# getting path to dataset
path = kagglehub.dataset_download("jockeroika/clouds-photos")


Transformers for input images

In [189]:

train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.RandomAutocontrast(),
    transforms.ToTensor(),
    transforms.Resize((64, 64)),
])

test_tranforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((64, 64)),
    ])

Preparing dataloader

In [190]:
dataset_train = ImageFolder(
    path + "/clouds_train",
    transform=train_transforms,   
)

dataloader_train = DataLoader(
    dataset_train,
    batch_size = 32,
    shuffle=True
)

dataset_test = ImageFolder(
    path +"/clouds_test",
    transform=test_tranforms
)

dataloader_test = DataLoader(
    dataset_test,
)

Convolutional Neural Networks

Setting seed

Dlaczego ELU a nie RELU - ELU  pomaga w problemie zanikających i wybuchających gradientów oraz  wymierających neuronów


Architektura sieci - artykuł cs231

![image](images/architecture.png)



Budowa sieci podstawowej:
    (Conv -> Elu)x2 -> MaxPool ->(Conv -> Elu)x2 -> MaxPool -> Conv->Elu -> MaxPool -> AdaptiveAvg -> DenseLayer


In [ ]:


class Net1(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.features_extractor = nn.Sequential(
            
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ELU(),
            # teraz mamy 32 x 64 x 64

            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ELU(),
            # teraz mamy 32 x 64 x 64

            nn.MaxPool2d(kernel_size=2),
            # teraz mamy 64 x 32 x 32

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ELU(),
            # teraz mamy 64 x 64 x 64
            
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ELU(),
            # teraz mamy 64 x 64 x 64
            
            nn.MaxPool2d(kernel_size=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ELU(),
            # teraz mamy 128 x 32 x 32

            nn.MaxPool2d(kernel_size=2),
            # teraz mamy 128 x 16 x 16 (jest to bardzo dużo wag - zwiększa ryzyko overfittingu)

            nn.AdaptiveAvgPool2d((1,1)),
            # adaptive avg pool przeciwdziała przeuczeniu się modelu

            nn.Flatten(),
            # teraz mamy 128 

            
        )
    
        # dense layer at the end
        self.classifier = nn.Linear(128, num_classes)
        
    def forward(self, x):
        x = self.features_extractor(x)
        x = self.classifier(x)
        return x
    


In [ ]:
class Net2(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.features_extractor = nn.Sequential(
            
            
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ELU(),
            # teraz mamy 32 x 64 x 64

            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ELU(),
            # teraz mamy 32 x 64 x 64

            nn.MaxPool2d(kernel_size=2),
            # teraz mamy 64 x 32 x 32


            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ELU(),
            # teraz mamy 64 x 64 x 64
            
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ELU(),
            # teraz mamy 64 x 64 x 64
            
            nn.MaxPool2d(kernel_size=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            #batchnormalization
            nn.BatchNorm2d(128),
            
            nn.ELU(),
            # teraz mamy 128 x 32 x 32

            nn.MaxPool2d(kernel_size=2),
            # teraz mamy 128 x 16 x 16 (jest to bardzo dużo wag - zwiększa ryzyko overfittingu)

            nn.AdaptiveAvgPool2d((1,1)),
            # adaptive avg pool przeciwdziała przeuczeniu się modelu

            nn.Flatten(),
            # teraz mamy 128 

            
        )
    
        # dense layer at the end
        self.classifier = nn.Linear(128, num_classes)
        
    def forward(self, x):
        x = self.features_extractor(x)
        x = self.classifier(x)
        return x


Batchnormalization pomaga sieci w uczeniu się (stabilizuje proces uczenia się naszej sieci)

In [ ]:

class Net3(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.features_extractor = nn.Sequential(
            
            
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ELU(),
            # teraz mamy 32 x 64 x 64

            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ELU(),
            # teraz mamy 32 x 64 x 64

            nn.MaxPool2d(kernel_size=2),
            # teraz mamy 32 x 32 x 32


            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ELU(),
            # teraz mamy 64 x 32 x 32
            
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ELU(),
            # teraz mamy 64 x 32 x 32
            
            nn.MaxPool2d(kernel_size=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ELU(),
            # teraz mamy 128 x 32 x 32

            nn.MaxPool2d(kernel_size=2),
            # teraz mamy 128 x 16 x 16 (jest to bardzo dużo wag - zwiększa ryzyko overfittingu)

            nn.AdaptiveAvgPool2d((1,1)),
            # adaptive avg pool przeciwdziała przeuczeniu się modelu

            nn.Flatten(),
            # teraz mamy 128 

            # dropout, żeby przeciwdziałać overfittingowi
            nn.Dropout(p=0.05),
            
        )

        self.classifier = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.features_extractor(x)
        x = self.classifier(x)
        return x


Dropout zapobiega przeuczaniu się sieci konwolucyjnej, lecz przy naszej sieci i małym zbiorze uczącym ma ona znikomy wpływ, lub czasem negatywny

In [209]:

net1 = Net1(num_classes=7)
net2 = Net2(num_classes=7)
net3 = Net3(num_classes=7)


In [218]:

criterion = nn.CrossEntropyLoss()
optimizer1 = optim.Adam(net1.parameters(), lr=0.001)
optimizer2 = optim.Adam(net2.parameters(), lr=0.001)
optimizer3 = optim.Adam(net3.parameters(), lr=0.001)

for epoch in range(10):
    for images, labels in dataloader_train:
        optimizer1.zero_grad()
        optimizer2.zero_grad()
        optimizer3.zero_grad()
        outputs1 = net1(images)
        outputs2 = net2(images)
        outputs3 = net3(images)
        loss1 = criterion(outputs1, labels)
        loss2 = criterion(outputs2, labels)
        loss3 = criterion(outputs3, labels)
        loss1.backward()
        optimizer1.step()
        loss2.backward()
        optimizer2.step()
        loss3.backward()
        optimizer3.step()

In [211]:


metric_precision = Precision(
    task = "multiclass",
    num_classes=7,
    average="macro"
    )

metric_recall = Recall(
    task = "multiclass",
    num_classes=7,
    average="macro"
)


In [219]:

import torch
from torchmetrics.classification import Accuracy

def eval_accuracy(net, dataloader, num_classes=7):
    net.eval()
    acc = Accuracy(task="multiclass", num_classes=num_classes)

    with torch.no_grad():
        for images, labels in dataloader:
            outputs = net(images)
            preds = outputs.argmax(dim=1)
            acc.update(preds, labels)
    return acc.compute().item()

a1 = eval_accuracy(net1, dataloader_test, 7)
a2 = eval_accuracy(net2, dataloader_test, 7)
a3 = eval_accuracy(net3, dataloader_test, 7)

print("net1 acc:", a1)
print("net2 acc:", a2)
print("net3 acc:", a3)



net1 acc: 0.7386831045150757
net2 acc: 0.8168724179267883
net3 acc: 0.7386831045150757
